# Create a simple ai assistant 
- This example we create a simple ai assistant and ask it a question about a sysml model.
- We can extract the returned python code.

In [1]:
from openai import OpenAI
import os
API_KEY=os.environ.get('OPENAI_API_KEY')

In [2]:
# Set up directories
BASE_DIR = os.getcwd() + "/" + "models"
BASE_DIR

'/usr/src/app/notebooks/simple_example/models'

In [3]:
%put file_content
# Python kernel cell
# Read the file and store its content in a variable
file_path = f'{BASE_DIR}/block.sysml'
with open(file_path, 'r') as file:
    file_content = file.read()
    print(file_content)

package block {
    import ScalarValues::*;
    
    attribute def Length;
    attribute def Width;
    attribute def Height;
    attribute def Volume;
    
    par def Block {
        attribute length: Length;
        attribute width: Width;
        attribute heigth: Height;
        attribute volume: Volume;
    }
    
    part block_1 {
        attribute i_length: Real = 2;
        attribute i_width: Real = 2;
        attribute i_heigth: Real = 2;
        attribute volume: Real = 0;
    }
    
    analysis def VolumeAnalysis {
        subject block_1: Block
    }
}


In [4]:
client = OpenAI(api_key=API_KEY,)
client

In [5]:
file_ids = []
# Assuming client.files.create correctly uploads the files and returns their file objects
model = client.files.create(file=open(f'{BASE_DIR}/block.sysml', "rb"), purpose='assistants')
file_ids.append(model.id)
# template = client.files.create(file=open(f'{BASE_DIR}/template.sysml', "rb"), purpose='assistants')
# file_ids.append(template.id)
print(file_ids)

['file-mX9xpjufwCRtp6M92Pr9krxi']


In [6]:
tools = [{"type": "retrieval"}]
tools

[{'type': 'retrieval'}]

In [7]:
# Pass file_ids directly to the assistant creation call
assistant = client.beta.assistants.create(
    name="SysMLv2-Example",
    instructions="You are a helpful assistant. Given a set of files, you extract the most interesting information and restructure it into python code to be executed as a part of a digital workflow.",
    model="gpt-4-1106-preview",
    tools=tools,  # Ensure tools is defined earlier or include the necessary tools here
    file_ids=file_ids  # Use the list directly without wrapping it in another list
)

assistant_id = assistant.id
print(assistant_id)

asst_aI6R7QsXXEjsvU0I2NMP8Z8O


In [8]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Discuss the block.sysml model and provide some example python code to evaluate the model?",
      "file_ids": file_ids
    }
  ]
)

thread_id = thread.id
thread_id

'thread_mpp1g4U1eZdqOBQzHpmVYEdT'

In [9]:
run = client.beta.threads.runs.create(
  thread_id=thread_id,
  assistant_id=assistant_id
)

run_id = run.id
run_id

'run_3weORQnKvJO83CCZHMfkStfH'

In [10]:
import time

# Start an infinite loop to continuously check the status
while True:
    # Retrieve the current status of the run
    run_status_response = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
    
    # Check if the response has the attribute or key for status; adjust based on the response structure
    if hasattr(run_status_response, 'status'):  # If the response is an object with attributes
        current_status = run_status_response.status
    elif 'status' in run_status_response:  # If the response is a dictionary
        current_status = run_status_response['status']
    else:
        print("Unable to find status in the response.")
        break
    
    print(f"Current status: {current_status}")
    
    # Break the loop if the run status is 'complete' or adjust according to the correct completion status
    if current_status == 'completed':
        print("Run is complete.")
        break
    elif current_status == 'failed':
        print("Run failed.")
        break
    
    # Sleep for a bit before checking the status again to avoid overwhelming the server
    time.sleep(2)

Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: in_progress
Current status: completed
Run is complete.


In [11]:
# Retrieve the list of messages for the specified thread_id (pause for a return here)
thread_messages = client.beta.threads.messages.list(thread_id=thread_id)
thread_messages

SyncCursorPage[Message](data=[Message(id='msg_a2fyH25XyVhl0OUdp0IjUTYU', assistant_id='asst_aI6R7QsXXEjsvU0I2NMP8Z8O', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='of file or use a local Python environment to parse and evaluate the SysML model.\n\nIf you have further details about the model or require assistance with a different task, please let me know!'), type='text')], created_at=1712156331, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_3weORQnKvJO83CCZHMfkStfH', status=None, thread_id='thread_mpp1g4U1eZdqOBQzHpmVYEdT'), Message(id='msg_CaEjdz4oetv9WSE7LgPSLYAz', assistant_id='asst_aI6R7QsXXEjsvU0I2NMP8Z8O', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='It appears that the file you uploaded is a SysML (Systems Modeling Language) model which goes by the name `block.sysml`. SysML is a general-purpose modeling language for systems engineering 

In [12]:
# Function to handle and print each content block in a message
def print_content_blocks(content_blocks):
    for block in content_blocks:
        # Assuming 'type' and 'text' are attributes of the block object
        if hasattr(block, 'type') and block.type == 'text':
            # Further assuming 'value' can be accessed directly from the 'text' attribute
            if hasattr(block.text, 'value'):
                text_value = block.text.value
                print(f"Content: {text_value}")
            else:
                print("Content: [Text block without accessible value]")
        else:
            # Handle other types of content blocks or print a placeholder
            print("Content: [Non-text content block]")

# Sample code to iterate through messages and print their content
for message in thread_messages.data:
    print(f"\nRole: {message.role}")
    
    # If 'content' is expected to be a list or similar iterable of content blocks
    if hasattr(message, 'content') and isinstance(message.content, list):
        print_content_blocks(message.content)
    else:
        # If 'content' is not a list, try to print it directly (or handle it as needed)
        print("Content: [Message content is not a list]")
    
    # Check for attached files
    if hasattr(message, 'file_ids') and message.file_ids:
        print("Attached Files:")
        for file_id in message.file_ids:
            print(f" - {file_id}")
    else:
        print("No attached files.")

    print("\n" + "-"*50)



Role: assistant
Content: of file or use a local Python environment to parse and evaluate the SysML model.

If you have further details about the model or require assistance with a different task, please let me know!
No attached files.

--------------------------------------------------

Role: assistant
Content: It appears that the file you uploaded is a SysML (Systems Modeling Language) model which goes by the name `block.sysml`. SysML is a general-purpose modeling language for systems engineering applications which supports the specification, analysis, design, verification, and validation of a broad range of systems and systems-of-systems.

Since your file cannot be accessed directly by the `myfiles_browser` tool, I am unable to read or analyze its contents, so I can't discuss the specifics of your `block.sysml` model.

However, I can guide you on how to process and evaluate a SysML model using Python in general. Python doesn't have native support for SysML, but you can use external 

In [13]:
import re

# Function to extract and print Python code blocks from the raw message data
def extract_python_code(raw_messages):
    python_code_blocks = []
    
    # Regular expression pattern to match Python code blocks
    python_code_pattern = re.compile(r"```python\n(.*?)```", re.DOTALL)
    
    for message in raw_messages:
        for content_block in message.content:
            # Assuming each content_block has a 'text' object with a 'value' attribute
            text_content = content_block.text.value
            
            # Search for Python code blocks within the text content
            matches = python_code_pattern.findall(text_content)
            python_code_blocks.extend(matches)
    
    return python_code_blocks

# Assuming 'thread_messages.data' is a list of messages as described in your raw thread message structure
raw_messages = thread_messages.data  # This should be the actual data structure containing your messages

# Extract Python code blocks
extracted_python_code = extract_python_code(raw_messages)

# Print extracted Python code blocks
for index, code in enumerate(extracted_python_code, start=1):
    print(f"Python Code Block {index}:\n{code}\n\n---\n")


Python Code Block 1:
from sysml_parser import parse_sysml
from sysml_evaluator import evaluate_model

# Load your SysML model
sysml_model_path = 'path_to_your_block.sysml'

# Parse the SysML model into a readable format for Python
model = parse_sysml(sysml_model_path)

# Evaluate the SysML model
evaluation_results = evaluate_model(model)

# Print the results
print("Evaluation of the SysML model:")
for result in evaluation_results:
    print(result)


---



In [14]:
# Lets clean up the assistnat and the files

# Delete the files first
client.files.delete(file_ids[0])

# Then delete the assistant
client.beta.assistants.delete(assistant_id)

AssistantDeleted(id='asst_aI6R7QsXXEjsvU0I2NMP8Z8O', deleted=True, object='assistant.deleted')